In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import networkx as nx
import freeman as fm

In [2]:
g = fm.load('marvel.gml')
g.label_nodes('name')
g.set_all_nodes(size=15, labpos='hover')
g.set_all_edges(color=(0, 0, 0, 0.5))

In [3]:
bc = nx.betweenness_centrality(g)

In [4]:
import pandas as pd
data = pd.DataFrame({
    'id' : [i for i in g.nodes],
    'Name': [g.nodes[i]['name'] for i in g.nodes],
    'ConnAmnt': [g.nodes[i]['amount'] for i in g.nodes],
    'Intermediation (r)': [i for i in bc.values()],
})

In [5]:
char_infos = pd.read_csv('../dataset/characters_stats.csv', index_col=0)
semi_df = pd.merge(data, char_infos, how='inner', on=['Name']) 

In [6]:
import numpy as np
semi_df = semi_df.replace('nan', np.nan)
semi_df = semi_df[semi_df['ConnAmnt'].notna()]
semi_df['ConnAmnt'] = pd.to_numeric(semi_df['ConnAmnt'])

In [7]:
semi_df = semi_df.drop_duplicates(subset=['Name'], keep='first')
semi_df = semi_df[semi_df['Alignment'] != 'neutral']
semi_df = semi_df.replace(['good', 'bad'], [0, 1])

In [8]:
semi_df['Intelligence (%)'] = semi_df['Intelligence']/semi_df['Total']
semi_df['Power (%)'] = semi_df['Power']/semi_df['Total']
semi_df['Strength (%)'] = semi_df['Strength']/semi_df['Total']
semi_df['Combat (%)'] = semi_df['Combat']/semi_df['Total']

In [9]:
# Existem muitos personagens nesses dados cuja soma Total é composta de 
# Intelligence  Strength    Speed    Durability   Power   Combat  Total
#     1            1          1         1           0       1       5
semi_df = semi_df[semi_df['Total'] != 5]

In [10]:
import statsmodels.api as sm

# model = sm.OLS(semi_df['Intermediation (r)'], semi_df[['Intelligence', 'Strength', 'Strength (%)', 
#                                                        'Power', 'Power (%)', 'Alignment']])
model = sm.OLS(semi_df['Intermediation (r)'], semi_df[['Intelligence', 'Strength', 'Power', 'Alignment']])
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:     Intermediation (r)   R-squared (uncentered):                   0.471
Model:                            OLS   Adj. R-squared (uncentered):              0.454
Method:                 Least Squares   F-statistic:                              28.22
Date:                Mon, 23 Nov 2020   Prob (F-statistic):                    8.90e-17
Time:                        22:56:07   Log-Likelihood:                          531.57
No. Observations:                 131   AIC:                                     -1055.
Df Residuals:                     127   BIC:                                     -1044.
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intelligence  7.942e-05   1.56e-05      5.105      0.000    4.86e-05       0.000
Strength      3.298e-06   1.25e-05      0.263      0.793   -2.15e-05    2.81e-05
Power        -1.776e-05   1.59e-05     -1.116      0.267   -4.92e-05    1.37e-05
Alignment       -0.0032      0.001     -3.560      0.001      -0.005      -0.001
==============================================================================
Omnibus:                       46.940   Durbin-Watson:                   1.491
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               94.347
Skew:                           1.575   Prob(JB):                     3.26e-21
Kurtosis:                       5.714   Cond. No.                         248.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [11]:
print(f"Coef:\n{result.params}\n")
print(f"std err:\n{result.bse}\n")
print(f"p-values:\n{result.pvalues}\n")

Coef:
Intelligence    0.000079
Strength        0.000003
Power          -0.000018
Alignment      -0.003217
dtype: float64

std err:
Intelligence    0.000016
Strength        0.000013
Power           0.000016
Alignment       0.000904
dtype: float64

p-values:
Intelligence    0.000001
Strength        0.792779
Power           0.266505
Alignment       0.000522
dtype: float64

